<a href="https://colab.research.google.com/github/stevenbowler/FineTuneLLaMa2/blob/master/llama_2_7b_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb)

# Fine Tune LLaMa 7B model with RAG

SB: 08/12/2023 updated to use 7b LLaMa model, to lower gpu and ram requirements while finetuning a RAG model that includes a very large open source rules and regs document chunked for consumption in a pinecone vector store.  
---

🚨 _Note that running this on CPU is sloooow. If running on Google Colab you can avoid this by going to **Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**. This should be included within the free tier of Colab._

---

We start by doing a `pip install` of all required libraries.

In [24]:
!pip install -qU \
  textract \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0 \
  pypdf==3.15.1

## Initializing the Hugging Face Embedding Pipeline

We begin by initializing the embedding pipeline that will handle the transformation of our docs into vector embeddings. We will use the `sentence-transformers/all-MiniLM-L6-v2` model for embedding.

In [26]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

Mounted at /content/drive


We can use the embedding model to create document embeddings like so:

In [27]:
docs = !source /content/drive/MyDrive/ColabStuff/2023-07-TMPPM.pdf


embeddings = embed_model.embed_documents(docs)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 1 doc embeddings, each with a dimensionality of 384.


SB: REPLACE, chunk out the TMHP file set as data variable in place of .... With our index and embedding process ready we can move onto the indexing process itself. For that, we'll need a dataset. We will use a set of Arxiv papers related to (and including) the Llama 2 research paper.

In [ ]:
# # original prep data for embed and index, below will create chunks from TMHP docs file
# from datasets import load_dataset

# data = load_dataset(
#     'jamescalam/llama-2-arxiv-papers-chunked',
#     split='train'
# )
# data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})

SB: Create chunks from TMHP document

In [28]:
# create text string from the pdf file
from pypdf import PdfReader

reader = PdfReader('/content/drive/MyDrive/ColabStuff/2023-07-TMPPM.pdf')
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [31]:
from langchain.docstore import DocumentStore

text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

# Create a list of documents.
documents = []

# Add a document to the list.
document = DocumentStore()
document.page_content = text
documents.append(document)

# Split the documents.
text_chunks = text_splitter.split_documents(documents)

ImportError: ignored

## Building the Vector Index

We now need to use the embedding pipeline to build our embeddings and store them in a Pinecone vector index. To begin we'll initialize our index, for this we'll need a [free Pinecone API key](https://app.pinecone.io/).

In [ ]:
from pydantic import env_settings
import os
import pinecone

# load env variables from Google Drive
# !source /content/drive/MyDrive/ColabStuff/env_vars.sh
env_vars = !source /content/drive/MyDrive/ColabStuff/env_vars.sh

# get API key from app.pinecone.io and environment from console
pinecone.init(
    # api_key=os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY',
    # environment=os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENV'
    api_key='',
    environment='gcp-starter'
)

Now we initialize the index.

In [ ]:
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

Now we connect to the index:

In [ ]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

We will embed and index the documents like so:

In [ ]:
data = data.to_pandas()

batch_size = 32

for i in range(0, len(data), batch_size):
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    texts = [x['chunk'] for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

## Initializing the Hugging Face Pipeline

The first thing we need to do is initialize a `text-generation` pipeline with Hugging Face transformers. The Pipeline requires three things that we must initialize first, those are:

* A LLM, in this case it will be `meta-llama/Llama-2-7b-chat-hf`.

* The respective tokenizer for the model.

We'll explain these as we get to them, let's begin with our model.

We initialize the model and move it to our CUDA-enabled GPU. Using Colab this can take 5-10 minutes to download and initialize the model.

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = ''
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


SB add model fine tuning

In [ ]:
# install fine tuner
!pip install autotrain-advanced
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 90.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 66.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 112.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

SB: Setup auto-trainer for Google Colab

In [ ]:
!autotrain setup --update-torch

2023-08-12 23:49:47.420225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


SB:HuggingFace credentials:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

--- Fine Tuning Command

!autotrain llm --train --project_name your_project_name --model TinyPixel/Llama-2-7B-bf16-sharded --data_path your_data_set --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 2 --num_train_epochs 3 --trainer sft --model_max_length 2048 --push_to_hub --repo_id your_repo_id

In [ ]:
!autotrain llm --train --project_name 'finetunellama2' --model model --data_path jamescalam/llama-2-arxiv-papers-chunked --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 2 --num_train_epochs 3 --trainer sft --model_max_length 2048 --push_to_hub --repo_id stevenbowler/FineTuneLLaMa2

2023-08-12 23:56:25.487214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='jamescalam/llama-2-arxiv-papers-chunked', train_split='train', valid_split=None, text_column='text', model='model', learning_rate=0.0002, num_train_epochs=3, train_batch_size=2, eval_batch_size=4, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, training_type='generic', train_on_inputs=False, logging_steps=-1, project_name='finetunellama2', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=True, use_int8=False, model_max_length=2048, repo_id='stevenbowler/FineTuneLLaMa2', use_int4=True, trai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The pipeline requires a tokenizer which handles the translation of human readable plaintext to LLM readable token IDs. The Llama 2 13B models were trained using the Llama 2 13B tokenizer, which we initialize like so:

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Now we're ready to initialize the HF pipeline. There are a few additional parameters that we must define here. Comments explaining these have been included in the code.

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

Confirm this is working:

In [ ]:
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])

Explain to me the difference between nuclear fission and fusion. Unterscheidung between nuclear fission and fusion: Nuclear fission is a process in which an atomic nucleus splits into two or more smaller nuclei, releasing energy in the process. Nuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus.
Nuclear fission is a process where an atomic nucleus splits into two or more smaller nuclei, releasing energy in the process. This process typically occurs when an atom's nucleus is bombarded with a high-energy particle, such as a neutron. When this happens, the nucleus can become unstable and break apart into lighter elements, releasing a large amount of energy in the process.
Nuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases energy, but it does so in a much more controlled and sustained manner than nuclear fission.

Now to implement this in LangChain

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
llm(prompt="Explain to me the difference between nuclear fission and fusion.")

" Unterscheidung between nuclear fission and fusion: Nuclear fission is a process in which an atomic nucleus splits into two or more smaller nuclei, releasing energy in the process. Nuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus.\nNuclear fission is a process where an atomic nucleus splits into two or more smaller nuclei, releasing energy in the process. This process typically occurs when an atom's nucleus is bombarded with a high-energy particle, such as a neutron. When this happens, the nucleus can become unstable and break apart into lighter elements, releasing a large amount of energy in the process.\nNuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases energy, but it does so in a much more controlled and sustained manner than nuclear fission. In nuclear fusion, the nuclei of two atoms are brought toget

We still get the same output as we're not really doing anything differently here, but we have now added **Llama 2 13B Chat** to the LangChain library. Using this we can now begin using LangChain's advanced agent tooling, chains, etc, with **Llama 2**.

## Initializing a RetrievalQA Chain

For **R**etrieval **A**ugmented **G**eneration (RAG) in LangChain we need to initialize either a `RetrievalQA` or `RetrievalQAWithSourcesChain` object. For both of these we need an `llm` (which we have initialized) and a Pinecone index — but initialized within a LangChain vector store object.

Let's begin by initializing the LangChain vector store, we do it like so:

In [ ]:
from langchain.vectorstores import Pinecone

text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

We can confirm this works like so:

In [ ]:
query = 'what makes llama 2 special?'

vectorstore.similarity_search(
    query,  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='Ricardo Lopez-Barquilla, Marc Shedroﬀ, Kelly Michelena, Allie Feinstein, Amit Sangani, Geeta\nChauhan,ChesterHu,CharltonGholson,AnjaKomlenovic,EissaJamil,BrandonSpence,Azadeh\nYazdan, Elisa Garcia Anzano, and Natascha Parks.\n•ChrisMarra,ChayaNayak,JacquelinePan,GeorgeOrlin,EdwardDowling,EstebanArcaute,Philomena Lobo, Eleonora Presani, and Logan Kerr, who provided helpful product and technical organization support.\n46\n•Armand Joulin, Edouard Grave, Guillaume Lample, and Timothee Lacroix, members of the original\nLlama team who helped get this work started.\n•Drew Hamlin, Chantal Mora, and Aran Mun, who gave us some design input on the ﬁgures in the\npaper.\n•Vijai Mohan for the discussions about RLHF that inspired our Figure 20, and his contribution to the\ninternal demo.\n•Earlyreviewersofthispaper,whohelpedusimproveitsquality,includingMikeLewis,JoellePineau,\nLaurens van der Maaten, Jason Weston, and Omer Levy.', metadata={'source': 'http://arxiv.org/pdf/230

Looks good! Now we can put our `vectorstore` and `llm` together to create our RAG pipeline.

In [ ]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

Let's begin asking questions! First let's try *without* RAG:

In [ ]:
llm('what is so special about llama 2?')

'\n nobody knows.\n\nBut the llama 2 has a secret: it\'s actually a highly advanced AI language model, capable of generating human-like text based on the input it receives. It can be trained on large datasets of text data and can learn to mimic the style and tone of any author or genre of writing.\nThe llama 2 is a tool for writers, artists, and anyone who wants to create content that is both creative and accurate. With its advanced language generation capabilities, the llama 2 can help you generate ideas, write articles, create poetry, and even translate text from one language to another.\nBut don\'t just take our word for it! Here are some examples of what the llama 2 can do:\n* Generate a poem in the style of William Shakespeare:\nOde to a Llama\nIn days of old, when tales were told\nOf knights and dragons, brave and bold\nThere lived a creature, oh so fair\nA llama, with a woolly mane so rare\n\n* Write an article on the history of llamas:\nLlamas have been around for thousands of 

Hmm, that's not what we meant... What if we use our RAG pipeline?

In [ ]:
rag_pipeline('what is so special about llama 2?')

{'query': 'what is so special about llama 2?',
 'result': ' Llama 2 is a collection of large language models (LLMs) that have been pretrained and fine-tuned for dialogue use cases. The models in Llama 2 outperform open-source chat models on most benchmarks and have been evaluated for helpfulness and safety. Unlike other publicly released pretrained LLMs, Llama 2 has been designed to be a suitable substitute for closed "product" LLMs like ChatGPT, BARD, and Claude.'}

This looks *much* better! Let's try some more.

In [ ]:
llm('what safety measures were used in the development of llama 2?')

"\n\nI'm looking for information on how the developers of Llama 2 ensured the safety of their users during the development process. Specifically, I'm interested in knowing about any safety measures that were implemented to protect users from potential risks or hazards associated with the use of the platform.\n\nHere are some possible answers:\n\n1. The developers of Llama 2 conducted thorough risk assessments to identify and mitigate any potential safety risks associated with the platform. This included identifying potential hazards such as data breaches, cyber attacks, and other security risks, and implementing appropriate safeguards to prevent these risks from occurring.\n2. The platform was designed with user privacy and security in mind, and the developers implemented various measures to protect user data and ensure that it is not compromised. For example, the platform may have implemented encryption techniques to protect user data, or implemented strict access controls to limit wh

Okay, it looks like the LLM with no RAG is less than ideal — let's stop embarassing the poor LLM and stick with RAG + LLM. Let's ask the same question to our RAG pipeline.

In [ ]:
rag_pipeline('what safety measures were used in the development of llama 2?')

{'query': 'what safety measures were used in the development of llama 2?',
 'result': ' The development of llama 2 included safety measures such as pre-training, fine-tuning, and model safety approaches. Additionally, the authors delayed the release of the 34B model due to a lack of time to sufficiently red team.'}

A reasonable answer from the RAG pipeline, but it doesn't contain much information — maybe we can ask more about this, like what is this _"red team"_ procedure that delayed the launch of the 34B model?

In [ ]:
rag_pipeline('what red teaming procedures were followed for llama 2?')

{'query': 'what red teaming procedures were followed for llama 2?',
 'result': " The paper describes the red teaming procedures used for Llama 2. These included creating prompts that might elicit unsafe or undesirable responses from the model, such as those based on sensitive topics or those that could potentially cause harm if the model were to respond inappropriately. The red teaming exercises were performed by a set of experts who evaluated the model's responses and provided feedback on its performance. The paper also mentions that multiple additional rounds of red teaming were performed over several months to measure the robustness of the model as it was released internally."}

Very interesting!

In [ ]:
rag_pipeline('how does the performance of llama 2 compare to other local LLMs?')

{'query': 'how does the performance of llama 2 compare to other local LLMs?',
 'result': ' The performance of llama 2 is compared to other local LLMs such as chinchilla and bard in the paper. Specifically, the authors report that llama 2 outperforms these other models on the series of helpfulness and safety benchmarks they tested. Additionally, the authors note that llama 2 appears to be on par with some of the closed-source models, at least on the human evaluations they performed.'}

### additional tests using the THMP document

In [ ]:
rag_pipeline('how many volumes are there in the Texas Medicaid Provider Procedures Manual?')

{'query': 'how many volumes are there in the Texas Medicaid Provider Procedures Manual?',
 'result': " I don't know the exact number of volumes in the Texas Medicaid Provider Procedures Manual as it might be subject to change and I don't have access to the most up-to-date version of the manual. However, I can suggest some possible sources where you may be able to find this information, such as the official website of the Texas Medicaid program or by contacting the Texas Health and Human Services Commission directly."}